#GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

#Installation and Importation

In [ ]:
pip install mediapipe

     |████████████████████████████████| 32.8 MB 1.4 MB/s 


In [ ]:
import cv2
import mediapipe as mp
import json
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np

#Loading of Data

Possible to select which videos should be treated by MediaPipe
- start: startline in the data table (first video that you want to treat)
- end: stopline in the data table

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1m2l52fMnXSO_SIp_LRnX-NTjt50RJhwYdJ4cCKX1xa8/edit#gid=0')
data = wb.worksheet('data').get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
start = 0 #startline in the data table
end = 100 #stopline in the data table
sub_df = df[start:end]
sub_df

,Name,Gender (W/M),"Fail, Zone or Top (0/1/2)",Competition,Climber,Country
310,c313.mp4,W,0,Salt Lake Boulder 1,Lucia Dörffel,GER


**Mediapipe**

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [ ]:
path = '/content/drive/MyDrive/Boulder Project/'
names = sub_df["Name"].values

In [ ]:
for idx in range(len(names)):
  path_in = path + "videos/" + names[idx]
  for ending in [".MOV", ".mp4"]:
    names[idx] = names[idx].replace(ending, "")
  path_out_landmarks = path + "json/" + names[idx] + "_landmarks.json"
  path_out_visibilities = path + "json/" + names[idx] + "_visibilities.json"

  cap = cv2.VideoCapture(path_in)
  landmarks = {"nose":[], "left_shoulder":[], "right_shoulder":[], "left_elbow":[], "right_elbow":[], 
             "left_wrist":[], "right_wrist":[],  "left_hip":[], "right_hip":[], "left_knee":[], "right_knee":[],
             "left_ankle":[],  "right_ankle":[], "left_heel":[], "right_heel":[]}
  visibilities = {"nose":[], "left_shoulder":[], "right_shoulder":[], "left_elbow":[], "right_elbow":[], 
             "left_wrist":[], "right_wrist":[],  "left_hip":[], "right_hip":[], "left_knee":[], "right_knee":[],
             "left_ankle":[],  "right_ankle":[], "left_heel":[], "right_heel":[]}

  with mp_pose.Pose(
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
      success, image = cap.read()
      if not success:
        print("End of video.")
        # If loading a video, use 'break' instead of 'continue'.
        break

      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = pose.process(image)

      image_hight, image_width, _ = image.shape
      if not results.pose_landmarks:
        continue
      landmarks["nose"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y))
      landmarks["left_shoulder"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y))
      landmarks["right_shoulder"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y))
      landmarks["left_elbow"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y))
      landmarks["right_elbow"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].y))
      landmarks["left_wrist"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y))
      landmarks["right_wrist"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y))
      landmarks["left_hip"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y))
      landmarks["right_hip"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y))
      landmarks["left_knee"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y))
      landmarks["right_knee"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].y))
      landmarks["left_ankle"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y))
      landmarks["right_ankle"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y))
      landmarks["left_heel"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].y))
      landmarks["right_heel"].append((results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].x,
                                      results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].y))
      
      visibilities["nose"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility)
      visibilities["left_shoulder"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility)
      visibilities["right_shoulder"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility)
      visibilities["left_elbow"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].visibility)
      visibilities["right_elbow"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].visibility)
      visibilities["left_wrist"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].visibility)
      visibilities["right_wrist"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].visibility)
      visibilities["left_hip"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].visibility)
      visibilities["right_hip"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].visibility)
      visibilities["left_knee"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].visibility)
      visibilities["right_knee"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].visibility)
      visibilities["left_ankle"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].visibility)
      visibilities["right_ankle"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].visibility)
      visibilities["left_heel"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].visibility)
      visibilities["right_heel"].append(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].visibility)
      
    cap.release()

  keys = visibilities.keys()
  for key in keys:
    visibilities[key] = np.mean(visibilities[key])

  with open(path_out_landmarks, "w") as outfile:
    json.dump(landmarks, outfile)
  with open(path_out_visibilities, "w") as outfile:
    json.dump(visibilities, outfile)
  print("Done: " + names[idx])

End of video.


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Done: c313
